<a href="https://colab.research.google.com/github/kavyajeetbora/nlp_rag/blob/master/end_to_end/02_local_llm_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`colab-xterm` allows you to open a terminal in a cell.

In [1]:
!pip install -q -U transformers python-dotenv langchain langchain_community langchain_huggingface accelerate bitsandbytes unstructured pdfminer randomname chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 27.9 M

In [2]:
import os
from glob import glob
import shutil
import randomname
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.retrievers import MultiQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

Load env variables:

In [3]:
if os.path.exists(".env"):
    os.remove(".env")

from google.colab import files
uploaded = files.upload()
if uploaded:
    if load_dotenv(".env"):
        print("Uploaded and Loaded Sucessfully")

from huggingface_hub import login
login(token=os.environ['HF_TOKEN'])

Saving .env to .env
Uploaded and Loaded Sucessfully


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Downloading the LLM model from HuggingFace

In [ ]:
model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

config = BitsAndBytesConfig(
    load_in_4bit = True
)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = model_id,
    quantization_config = config,
    low_cpu_mem_usage=True
    #attn_implementation="flash_attention_2", # if you have an ampere GPU
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, top_k=50, temperature=0.1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

In [ ]:
model_size = model.get_memory_footprint()/1e9
print(f"Model size of {model_id} : {model_size:.2f} GB")

Model size of microsoft/Phi-3-mini-4k-instruct : 4.02 GB


In [ ]:
num_params = model.num_parameters()/1e9
print(f"Number of parameters: {num_params:.2} Billon")

Number of parameters: 3.8 Billon


# Running Phi-4 model locally

first create the langchain llm pipeline:

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
query = "Explain the concept of quantum entanglement in simple terms."
expected_output =  "Quantum entanglement is a phenomenon where two particles become interconnected, so the state of one instantly influences the state of the other, no matter how far apart they are."

llm.invoke(query)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'Explain the concept of quantum entanglement in simple terms. Quantum entanglement is a phenomenon in quantum physics where two or more particles become linked in such a way that the state of one particle instantly influences the state of the other, no matter how far apart they are. This means that if you measure the state of one particle, you can instantly know the state of the other particle, even if they are light-years apart. This is a strange and counterintuitive concept, as it seems to violate the laws of classical'

## Let's build a local RAG System

### Ingest the data



In [ ]:
!wget -q https://www.niti.gov.in/sites/default/files/2023-02/Annual-Report-2022-2023-English_1.pdf -O annual_report.pdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [ ]:
doc_path = "annual_report.pdf"

if os.path.exists(doc_path):
    loader = PyPDFLoader(doc_path)
    data = loader.load()
    print("PDF is loaded")
else:
    print("PDF file not found")

PDF is loaded


In [ ]:
data[10]

Document(metadata={'source': 'annual_report.pdf', 'page': 10}, page_content='MoH&FW Ministry of Health and Family Welfare\nMoHUA Ministry of Housing and Urban Affairs\nMoSPI Ministry of Statistics and Programme Implementation\nNAS National Achievement Survey\nNDC Nationally Determined Contributions\nNEP National Education Policy\nNER North-Eastern Region\nNFHS National Family Health Survey\nNILERD National Institute of Labour Economics Research and Development\nNITI National Institution for Transforming India\nNMP National Monetisation Pipeline\nNOTP National Organ Transplant Programme\nOOMF Output–Outcome Monitoring Framework\nOoSC Out-of-School Children\nPLI Production Linked Incentive\nPMJAY Pradhan Mantri Jan Arogya Yojana\nPSHICMI Policy and Strategy for Health Insurance Coverage of India\nSATH-E Sustainable Action for Transforming Human Capital-Education\nSC-NEC Sub-Committee of National Executive Committee\nSDG Sustainable Development Goals\nSECI State Energy & Climate Index\nSE

### Chunking the information

In [ ]:
## Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300
)

chunks = text_splitter.split_documents(data)
print(f"The documents has been splitted into {len(chunks)} chunks")

The documents has been splitted into 386 chunks


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda:0'},
)

create the vector database

In [ ]:
def create_vector_database(chunks: list,name="vector-db"):

    os.makedirs("db", exist_ok=True)

    random_suffix = randomname.get_name()

    persistent_directory = f"db/chroma-{name}-({random_suffix})"

    ## If already there, delete and create a new one

    for folder in glob(f"db/chroma-{name}*"):
        if os.path.exists(folder):
            shutil.rmtree(folder)

    os.mkdir(persistent_directory)

    vector_db = Chroma.from_documents(
        documents = chunks,
        collection_name = name,
        embedding = embeddings,
        persist_directory = persistent_directory
    )

    return vector_db

In [ ]:
vector_db = create_vector_database(chunks=chunks, name="annual-report")

## Retrieval

In [ ]:
prompt_template = '''You are an AI language model assistant.
Generate 1-5 sub-questions or alternate versions of the given user question to
retrieve relevant documents from a vector database.
Break down questions with multiple concepts into distinct sub-questions.
Provide these alternative questions separated by newlines between XML tags.

Original question: {question}
'''

prompt = PromptTemplate(template = prompt_template, input_variables=["question"])

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    retriever = vector_db.as_retriever(),
    llm = llm,
    prompt = prompt
)

In [ ]:
chat_prompt_template = '''
You are a helpful and friendly assistant.
Answer the user's questions based solely on the context of the provided.
{context}
If the information is not available in the context, respond with "I don't know."

User: {question}
Assistant:
'''

chat_prompt = ChatPromptTemplate.from_template(chat_prompt_template)

## Define the RAG chain

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
%%time

query = 'What is the full form of RSNA ?'

rag_chain.invoke(query)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (11053 > 4096). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 346.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 121.06 MiB is free. Process 3066 has 14.63 GiB memory in use. Of the allocated memory 12.58 GiB is allocated by PyTorch, and 1.91 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)